# Flipside Crypto SDK
- Author: Steffan Rees
- 29/08/2022

## Resources
- https://github.com/FlipsideCrypto/sdk
- https://docs.flipsidecrypto.com/shroomdk-sdk/getting-started
- https://sdk.flipsidecrypto.xyz/shroomdk
- https://app.flipsidecrypto.com/velocity
- https://teamflipside.notion.site/teamflipside/SQL-Basics-8b3a10aa657f4b84bc80229f4619b0ea
- https://www.youtube.com/watch?v=VB0ZlZ_Hr_A
- https://github.com/FlipsideCrypto/sdk/tree/main/examples/python

In [1]:
from datetime import datetime
import math
import toml
from shroomdk import ShroomDK

In [2]:
# Read config file and import configurations
config = toml.load('../config.toml')
SHROOMDK_API = config.get('shroomdk').get('shroomdk_api')

In [3]:
# Initialise ShroomDK
sdk = ShroomDK(SHROOMDK_API)

In [4]:
# Constants
# The max number of rows a query can return and cache is 1M.
# However, only 100k rows/records can be returned in a single http call. 
MAX_QUERY_LIMIT = 1000000
PAGE_SIZE = 100000

### Example: get all Solana swaps in the last 15 days
https://www.youtube.com/watch?v=bD1ioDw41aw

In [5]:
# First need to get the total number of swaps and start and end dates
query_count = """
SELECT
  COUNT(1) as swap_count,
  GETDATE() as end_time,
  GETDATE() - interval '15 days' as start_time
FROM
  solana.fact_swaps
WHERE
  succeeded = true
  AND block_timestamp BETWEEN GETDATE() - interval '15 days'
  AND GETDATE()
"""

In [6]:
qrs_count = sdk.query(query_count)
qrs_count

QueryResultSet(query_id=None, status='finished', columns=['SWAP_COUNT', 'END_TIME', 'START_TIME'], column_types=['number', 'timestamp', 'timestamp'], rows=[[984348, '2022-08-29 20:33:04.164 +0000', '2022-08-14 20:33:04.164 +0000']], run_stats=QueryRunStats(started_at=datetime.datetime(2022, 8, 29, 20, 33, 4), ended_at=datetime.datetime(2022, 8, 29, 20, 33, 7), elapsed_seconds=3, record_count=1), records=[{'swap_count': 984348, 'end_time': '2022-08-29 20:33:04.164 +0000', 'start_time': '2022-08-14 20:33:04.164 +0000'}], error=None)

In [7]:
# Total number of Solana swaps over the last 15 days
count = qrs_count.records[0]["swap_count"]
count

984348

In [8]:
start_time = qrs_count.records[0]["start_time"]
start_time

'2022-08-14 20:33:04.164 +0000'

In [9]:
end_time = qrs_count.records[0]["end_time"]
end_time

'2022-08-29 20:33:04.164 +0000'

In [10]:
query_runs = math.ceil(count/MAX_QUERY_LIMIT)
query_runs

1

In [11]:
all_records = []
start_recorded_time = datetime.now()
for query_run in range(query_runs):
    offset = query_run * MAX_QUERY_LIMIT
    query = f"""
    SELECT
      *
    FROM
      solana.fact_swaps
    WHERE
      succeeded = true
      AND block_timestamp BETWEEN '{start_time}' AND '{end_time}'
    ORDER BY
      block_timestamp DESC
    LIMIT
      {MAX_QUERY_LIMIT}
    OFFSET
      {offset}
    """
    for page_number in range(1, int(round(MAX_QUERY_LIMIT/PAGE_SIZE))+1):
        print(f"Querying {PAGE_SIZE} records from offset {offset} @ page {page_number}")
        qrs = sdk.query(query, page_number=page_number, page_size=PAGE_SIZE, ttl_minutes=1)
        if qrs.records is None:
            continue
        all_records.extend(qrs.records)
        print(f"Records returned thus far: {len(all_records)}")
        if all_records == count:
            break
    
print(f"Query took {datetime.now() - start_recorded_time} in total to run")
print(f"Total number of records returned: {len(all_records)}")

Querying 100000 records from offset 0 @ page 1
Records returned thus far: 100000
Querying 100000 records from offset 0 @ page 2
Records returned thus far: 200000
Querying 100000 records from offset 0 @ page 3
Records returned thus far: 300000
Querying 100000 records from offset 0 @ page 4
Records returned thus far: 400000
Querying 100000 records from offset 0 @ page 5
Records returned thus far: 500000
Querying 100000 records from offset 0 @ page 6
Records returned thus far: 600000
Querying 100000 records from offset 0 @ page 7
Records returned thus far: 700000
Querying 100000 records from offset 0 @ page 8
Records returned thus far: 800000
Querying 100000 records from offset 0 @ page 9
Records returned thus far: 900000
Querying 100000 records from offset 0 @ page 10
Records returned thus far: 984348
Query took 0:03:00.342871 in total to run
Total number of records returned: 984348


In [12]:
all_records[0]

{'swap_program': 'raydium v4',
 'block_timestamp': '2022-08-29 07:16:28.000',
 'block_id': 148163042,
 'tx_id': '2vm2CFV9zUT6i6kPU7xZgwaBypCutTpp7gS4as9sqs3yjfVc4aJMHTjuv9p64MMGtDyLZCHPNobXUeQxLv3auWDC',
 'succeeded': True,
 'swapper': 'GaLUQAjCwxGgTDG3ixy4MiFVBBx3txNupQWpcCn8Wsbw',
 'swap_from_amount': 4000,
 'swap_from_mint': 'DFL1zNkaGPWm1BqAVqRjCZvHmwTFrEaJtbzJWgseoNJh',
 'swap_to_amount': 7.778569,
 'swap_to_mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'}

## Adapted example: get all UniswapV2 swaps in the last 15 days

In [13]:
# First need to get the total number of swaps and start and end dates
query_count = """
SELECT
  COUNT(1) as swap_count,
  GETDATE() as end_time,
  GETDATE() - interval '15 days' as start_time
FROM
  ethereum.core.ez_dex_swaps
WHERE
  platform = 'uniswap-v2'
  AND event_name = 'Swap'
  AND block_timestamp BETWEEN GETDATE() - interval '15 days'
  AND GETDATE()
"""

In [14]:
qrs_count = sdk.query(query_count)
qrs_count

QueryResultSet(query_id=None, status='finished', columns=['SWAP_COUNT', 'END_TIME', 'START_TIME'], column_types=['number', 'timestamp', 'timestamp'], rows=[[1196050, '2022-08-29 20:36:21.226 +0000', '2022-08-14 20:36:21.226 +0000']], run_stats=QueryRunStats(started_at=datetime.datetime(2022, 8, 29, 20, 36, 21), ended_at=datetime.datetime(2022, 8, 29, 20, 36, 23), elapsed_seconds=2, record_count=1), records=[{'swap_count': 1196050, 'end_time': '2022-08-29 20:36:21.226 +0000', 'start_time': '2022-08-14 20:36:21.226 +0000'}], error=None)

In [15]:
# Total number of UniswapV2 swaps over the last 15 days
count = qrs_count.records[0]["swap_count"]
count

1196050

In [16]:
start_time = qrs_count.records[0]["start_time"]
start_time

'2022-08-14 20:36:21.226 +0000'

In [17]:
end_time = qrs_count.records[0]["end_time"]
end_time

'2022-08-29 20:36:21.226 +0000'

In [18]:
query_runs = math.ceil(count/MAX_QUERY_LIMIT)
query_runs

2

In [19]:
all_records = []
start_recorded_time = datetime.now()
for query_run in range(query_runs):
    offset = query_run * MAX_QUERY_LIMIT
    query = f"""
    SELECT
      *
    FROM
      ethereum.core.ez_dex_swaps
    WHERE
      platform = 'uniswap-v2'
      AND event_name = 'Swap'
      AND block_timestamp BETWEEN '{start_time}' AND '{end_time}'
    ORDER BY
      block_timestamp DESC
    LIMIT
      {MAX_QUERY_LIMIT}
    OFFSET
      {offset}
    """
    for page_number in range(1, int(round(MAX_QUERY_LIMIT/PAGE_SIZE))+1):
        print(f"Querying {PAGE_SIZE} records from offset {offset} @ page {page_number}")
        qrs = sdk.query(query, page_number=page_number, page_size=PAGE_SIZE, ttl_minutes=1)
        if qrs.records is None:
            continue
        all_records.extend(qrs.records)
        print(f"Records returned thus far: {len(all_records)}")
        if all_records == count:
            break
    
print(f"Query took {datetime.now() - start_recorded_time} in total to run")
print(f"Total number of records returned: {len(all_records)}")

Querying 100000 records from offset 0 @ page 1
Records returned thus far: 100000
Querying 100000 records from offset 0 @ page 2
Records returned thus far: 200000
Querying 100000 records from offset 0 @ page 3
Records returned thus far: 300000
Querying 100000 records from offset 0 @ page 4
Records returned thus far: 400000
Querying 100000 records from offset 0 @ page 5
Records returned thus far: 500000
Querying 100000 records from offset 0 @ page 6
Records returned thus far: 600000
Querying 100000 records from offset 0 @ page 7
Records returned thus far: 700000
Querying 100000 records from offset 0 @ page 8
Records returned thus far: 800000
Querying 100000 records from offset 0 @ page 9
Records returned thus far: 900000
Querying 100000 records from offset 0 @ page 10
Records returned thus far: 1000000
Querying 100000 records from offset 1000000 @ page 1
Records returned thus far: 1100000
Querying 100000 records from offset 1000000 @ page 2
Records returned thus far: 1196050
Querying 100

In [20]:
all_records[0]

{'block_number': 15435965,
 'block_timestamp': '2022-08-29 19:35:55.000',
 'tx_hash': '0xf9de715461264cd71edc63406f54084136ef45e2f2aea3bc1d2d014118de70e5',
 'origin_function_signature': '0x5ae401dc',
 'origin_from_address': '0xfc6ff36aa0e00a0e061bbe75828523226340d0dd',
 'origin_to_address': '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45',
 'contract_address': '0x632658ade70951cadc6cd116b966c51f89ed522d',
 'pool_name': None,
 'event_name': 'Swap',
 'amount_in': 0.1,
 'amount_in_usd': None,
 'amount_out': 5474789920708,
 'amount_out_usd': None,
 'sender': '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45',
 'tx_to': '0xfc6ff36aa0e00a0e061bbe75828523226340d0dd',
 'event_index': 224,
 'platform': 'uniswap-v2',
 'token_in': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
 'token_out': '0x349d13a1dd532b1cadb8676e55b34b8427a563c2',
 'symbol_in': 'WETH',
 'symbol_out': None,
 '_log_id': '0xf9de715461264cd71edc63406f54084136ef45e2f2aea3bc1d2d014118de70e5-224',
 '_inserted_timestamp': '2022-08-29 19:37:17.